In [2]:
import pandas as pd
import numpy as np

In [10]:
data=pd.read_csv("data.csv")
data.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,2.26,Ideal,G,SI2,61.9,57.0,8.44,8.36,5.20,12831
1,2.43,Very Good,H,SI2,63.2,57.0,8.56,8.50,5.39,16170
2,0.80,Premium,F,SI2,61.0,57.0,6.03,6.01,3.67,2797
3,0.40,Ideal,F,I1,63.3,60.0,4.68,4.64,2.95,630
4,0.31,Ideal,G,VS2,61.6,55.0,4.39,4.37,2.70,698


In [11]:
data=pd.get_dummies(data=data,columns=['cut','color','clarity'])

In [12]:
data.columns

Index(['carat', 'depth', 'table', 'x', 'y', 'z', 'price', 'cut_Fair',
       'cut_Good', 'cut_Ideal', 'cut_Premium', 'cut_Very Good', 'color_D',
       'color_E', 'color_F', 'color_G', 'color_H', 'color_I', 'color_J',
       'clarity_I1', 'clarity_IF', 'clarity_SI1', 'clarity_SI2', 'clarity_VS1',
       'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2'],
      dtype='object')

In [13]:
columns=['carat', 'depth', 'table', 'x', 'y', 'z', 'cut_Fair',
       'cut_Good', 'cut_Ideal', 'cut_Premium', 'cut_Very Good', 'color_D',
       'color_E', 'color_F', 'color_G', 'color_H', 'color_I', 'color_J',
       'clarity_I1', 'clarity_IF', 'clarity_SI1', 'clarity_SI2', 'clarity_VS1',
       'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2']
dictio={}
for col in columns:
    dictio[col]=[np.mean(data[col]),np.std(data[col])]
    data[col] = (data[col] - np.mean(data[col])) / np.std(data[col]) 
data.head()

,carat,depth,table,x,y,z,price,cut_Fair,cut_Good,cut_Ideal,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,3.081321,0.106170,-0.205764,2.411028,2.276274,2.339667,12831,-0.173938,-0.314915,1.227461,...,-0.333356,-0.234871,-0.118155,-0.186229,-0.564152,2.203499,-0.423692,-0.540520,-0.269537,-0.323005
1,3.439708,1.013798,-0.205764,2.517870,2.397719,2.607386,16170,-0.173938,-0.314915,-0.814690,...,-0.333356,-0.234871,-0.118155,-0.186229,-0.564152,2.203499,-0.423692,-0.540520,-0.269537,-0.323005
2,0.003405,-0.522188,-0.205764,0.265287,0.237737,0.183828,2797,-0.173938,-0.314915,-0.814690,...,-0.333356,-0.234871,-0.118155,-0.186229,-0.564152,2.203499,-0.423692,-0.540520,-0.269537,-0.323005
3,-0.839859,1.083616,1.138849,-0.936684,-0.950686,-0.830684,630,-0.173938,-0.314915,1.227461,...,-0.333356,-0.234871,8.463460,-0.186229,-0.564152,-0.453824,-0.423692,-0.540520,-0.269537,-0.323005
4,-1.029593,-0.103283,-1.102173,-1.194885,-1.184901,-1.182945,698,-0.173938,-0.314915,1.227461,...,-0.333356,-0.234871,-0.118155,-0.186229,-0.564152,-0.453824,-0.423692,1.850071,-0.269537,-0.323005


In [14]:
from h2o.automl import H2OAutoML

In [15]:
import h2o
h2o.init(nthreads = -1, max_mem_size = 4)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,1 hour 17 mins
H2O cluster timezone:,Europe/Madrid
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.11
H2O cluster version age:,3 days
H2O cluster name:,H2O_from_python_alvarorivas_r23x09
H2O cluster total nodes:,1
H2O cluster free memory:,3.432 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [16]:
datah2o=h2o.H2OFrame(data)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [17]:
y_columns="price"
x_columns=['carat', 'depth', 'table', 'x', 'y', 'z', 'cut_Fair',
       'cut_Good', 'cut_Ideal', 'cut_Premium', 'cut_Very Good', 'color_D',
       'color_E', 'color_F', 'color_G', 'color_H', 'color_I', 'color_J',
       'clarity_I1', 'clarity_IF', 'clarity_SI1', 'clarity_SI2', 'clarity_VS1',
       'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2']

In [18]:
train, test=datah2o.split_frame(ratios = [.8])

In [19]:
aml = H2OAutoML(max_runtime_secs= 180,max_models= 15, seed= 1, nfolds=0)
aml.train(x = x_columns, y = y_columns, training_frame = train, validation_frame=test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [20]:
lb= aml.leaderboard
lb

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_1_AutoML_20191209_164609,309393,556.231,309393,271.159,0.0981901
GBM_1_AutoML_20191209_164609,320189,565.853,320189,287.041,0.106467
GBM_4_AutoML_20191209_164609,326643,571.527,326643,276.897,0.0985912
XGBoost_2_AutoML_20191209_164609,334305,578.191,334305,270.013,0.0945257
GBM_3_AutoML_20191209_164609,335915,579.581,335915,289.916,0.108116
XGBoost_3_AutoML_20191209_164609,340010,583.104,340010,298.383,0.115867
GBM_2_AutoML_20191209_164609,346222,588.407,346222,287.696,0.10933
XRT_1_AutoML_20191209_164609,361139,600.949,361139,285.416,0.103089
DRF_1_AutoML_20191209_164609,363390,602.818,363390,285.214,0.102767
GBM_5_AutoML_20191209_164609,387896,622.813,387896,289.696,nan


In [22]:
datatest=pd.read_csv("test.csv")
datatest.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.56,Ideal,I,VS2,62.1,54.0,5.30,5.33,3.30
1,1,1.05,Ideal,G,VS2,61.9,56.0,6.56,6.52,4.05
2,2,0.50,Premium,E,VS2,61.5,56.0,5.11,5.07,3.13
3,3,0.80,Ideal,F,VS1,62.8,56.0,5.91,5.96,3.73
4,4,0.54,Ideal,G,VS1,61.4,57.0,5.23,5.20,3.20


In [23]:
datatest=pd.get_dummies(data=datatest,columns=['cut','color','clarity'])

In [24]:
for col in columns:
    datatest[col] = (datatest[col] - dictio[col][0] / dictio[col][1])
datatest.head()

,id,carat,depth,table,x,y,z,cut_Fair,cut_Good,cut_Ideal,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0,-1.123123,18.989094,28.246583,0.196484,0.354288,-1.687367,-0.173938,-0.314915,0.18531,...,0.666644,-0.234871,-0.118155,-0.186229,-0.564152,-0.453824,-0.423692,0.45948,-0.269537,-0.323005
1,1,-0.633123,18.789094,30.246583,1.456484,1.544288,-0.937367,-0.173938,-0.314915,0.18531,...,-0.333356,-0.234871,-0.118155,-0.186229,-0.564152,-0.453824,-0.423692,0.45948,-0.269537,-0.323005
2,2,-1.183123,18.389094,30.246583,0.006484,0.094288,-1.857367,-0.173938,-0.314915,-0.81469,...,-0.333356,-0.234871,-0.118155,-0.186229,-0.564152,-0.453824,-0.423692,0.45948,-0.269537,-0.323005
3,3,-0.883123,19.689094,30.246583,0.806484,0.984288,-1.257367,-0.173938,-0.314915,0.18531,...,-0.333356,-0.234871,-0.118155,-0.186229,-0.564152,-0.453824,0.576308,-0.54052,-0.269537,-0.323005
4,4,-1.143123,18.289094,31.246583,0.126484,0.224288,-1.787367,-0.173938,-0.314915,0.18531,...,-0.333356,-0.234871,-0.118155,-0.186229,-0.564152,-0.453824,0.576308,-0.54052,-0.269537,-0.323005


In [25]:
datah2otest=h2o.H2OFrame(datatest)
len(datah2otest)

Parse progress: |█████████████████████████████████████████████████████████| 100%


13485

In [26]:
predfinal = aml.leader.predict(datah2otest)
pdpred = h2o.as_list(predfinal)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [27]:
len(pdpred)==len(datatest)

True

In [28]:
predictions=pd.DataFrame({'id':datatest['id'],'price':pdpred['predict']})

In [29]:
predictions.to_csv('submition2.csv',index=False)